In [5]:
import pandas as pd
import requests
from datetime import datetime
import time
import os
import json
import psycopg2

from dotenv import load_dotenv

from bs4 import BeautifulSoup
pd.set_option('display.max_rows', None)

load_dotenv()

True

## Helper Functions

In [102]:
def coleta_html(link):
    headers = {'user-agent': 'Mozilla/5.0'}
    resposta = requests.get(link,
                            headers= headers
                            )
    proposta_soup = BeautifulSoup(resposta.text)

    return proposta_soup


def coleta_html_detalhado(link, nome_vereador):
    #print('1')
    if verifica_link(link, nome_vereador):
        return 0
    else:
        headers = {'user-agent': 'Mozilla/5.0'}
        resposta = requests.get(link,
                                headers= headers
                                )
        proposta_soup = BeautifulSoup(resposta.text)

        return proposta_soup
        

def conecta_banco_prefeitura():
    dbname = os.getenv("DBNAME")
    user = os.getenv("USER")
    password = os.getenv("PASSWORD")
    host = os.getenv("HOST")
    port = os.getenv("PORT")

    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    #cur = conn.cursor()

    return conn




def verifica_link(link, nome_vereador):

    conn = conecta_banco_prefeitura()
    cur = conn.cursor()

    cur.execute("SELECT link FROM links_proprosituras_vereadores WHERE nome_vereador =  %s", (nome_vereador,))

    results = cur.fetchall()

    df_teste = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])

    conn.commit()
    cur.close()
    conn.close()

    if link in list(df_teste['link']):
        return True 
    else:
        return False

def insere_link(link, nome):
    conn = conecta_banco_prefeitura()
    cur = conn.cursor()

    cur.execute("INSERT INTO links_proprosituras_vereadores (nome_vereador, link) VALUES (%s, %s);", (nome, link))
    conn.commit()
    cur.close()
    conn.close()

    

    

    


In [ ]:
headers = {'user-agent': 'Mozilla/5.0'}
resposta = requests.get('https://www.camarabotucatu.sp.gov.br/Vereador',
                        headers= headers
                        )

## Pegando o Nome e o Partido dos Vereadores

In [ ]:
text_estruturado = BeautifulSoup(resposta.text)
#text_estruturado

In [ ]:
lista_vereadores = text_estruturado.find_all('h2')

In [ ]:
lista_vereadores

In [ ]:
# Captando o nome e o partido dos vereadores
tags_li = text_estruturado.find_all('span')
lista_partidos = []
for i in range(len(tags_li)):
    if tags_li[i].find_all(['li']):
        if tags_li[i].find_all('a'):
            lista_partidos.append( tags_li[i].text.replace('\n', '') )


nomes_vereadores = []
for i in range(len(lista_vereadores)):
    #print(i)
    nomes_vereadores.append(  lista_vereadores[i].contents[1].text.split('-')[1].strip()  )


## Captando as imagens dos Vereadores

In [ ]:
# import re

# teste = []
# teste2 = []

# x = text_estruturado.find_all('div', {'class': 'col-md-3'})
# for i in range(len(x)):
#     if x[i].find('div', {'class':'img'}):
#         teste.append(x[i].find('div', {'class':'img'}))
#         #print(teste)
# for j in range(len(teste)):
#     if teste[j].find('img'):
#         teste2.append(teste[j].find_all('img', {'src': re.compile('\.jpg$', re.IGNORECASE)}))


In [ ]:
# save_folder = "../img"
# for img_tag in teste2:
#     src = img_tag[0]['src']
#     alt = img_tag[0]['alt'].split('-')[1].strip()
#     img_url = f"https://www.camarabotucatu.sp.gov.br/{src}" 
#     filename = os.path.join(save_folder, f"{alt}.jpg")

#     response = requests.get(img_url)
#     with open(filename, 'wb') as f:
#         f.write(response.content)

In [ ]:
df_vereadores = pd.DataFrame()
lista_partidos

df_vereadores['nome_vereador'] = nomes_vereadores
df_vereadores['partido_vereador'] = lista_partidos

In [ ]:
df_vereadores

## Para Cada Vereador trazer tabela do que ja fez em cada Ano

In [ ]:
# links_vereadores = []
# for i in range(len(lista_vereadores)):
#     links_vereadores.append(lista_vereadores[i].a.get('href'))

# df_vereadores['link'] = links_vereadores

In [ ]:
# df_final = pd.DataFrame(columns=['Tipo', 'Quantidade','Ano'])

# for i in range(len(df_vereadores)):

#     print(i)

#     #pega o nome do vereador
#     nome_atual_vereador = df_vereadores.iloc[i][:i+1]['nome_vereador']


#     #pega o link do vereador
#     #df_vereadores.iloc[i][:i+1]['link']


#     headers = {'user-agent': 'Mozilla/5.0'}
#     resposta_vereador = requests.get(df_vereadores[:i+1]['link'].iloc[-1],
#                         headers= headers
#                         )
    
#     vereador_estruturado = BeautifulSoup(resposta_vereador.text)
#     tabela_documentos = vereador_estruturado.find_all('table')

#     # Pegando a linha da tabela do site que contém os anos de atuação dos vereadores
#     lista_anos = []

#     for i in range(len(tabela_documentos)):
#         if tabela_documentos[i].find_all('caption')[0].text == 'Documentos':
#             tabela_documentos = tabela_documentos[i]


#     for i in range(len( tabela_documentos.find_all('th', {'class': 'text-right'}) ) ):
#         if tabela_documentos.find_all('th', {'class': 'text-right'})[i].text:
#             lista_anos.append(tabela_documentos.find_all('th', {'class': 'text-right'})[i].text)
#     lista_anos = sorted([*set(lista_anos)])
#     if 'Total' in lista_anos:
#         lista_anos.remove('Total')
#     #lista_anos


#     # Pegando todas as proposituras do vereador em cada tipo e alocando em um dataframe
#     tbody = tabela_documentos.find_all('tbody')[0].find_all('tr')#[0]#.find_all('td')
#     list_dict = []
#     df_teste = pd.DataFrame(columns=['Tipo', 'Quantidade'])

#     for i in range(len(tbody)):
#         for j in range(len( tbody[i].find_all('td') )):
#             list_dict.append(tbody[i].find_all('td')[j].text)
#         #print(list_dict)
        
#         #list_aux = list_dict
#         #list_aux.remove(list_aux[0])
#         df_novo = pd.DataFrame({'Tipo': [list_dict[0]]*(len(list_dict)-2), 'Quantidade': list_dict[1:-1], 'Ano': lista_anos,'Vereador': nome_atual_vereador})
#         df_teste = pd.concat([df_teste, df_novo], ignore_index=True)

#         list_dict = []
#     df_final = pd.concat([df_final,df_teste])
#     df_final = df_final.loc[df_final['Tipo'] != 'Total']
    
    

In [ ]:
# df_final['Quantidade'] = df_final['Quantidade'].apply(lambda x: 0 if x == '-' else x)
# df_final.head()

In [ ]:
# df_final['Quantidade'] = df_final['Quantidade'].astype('int')
# df_final['Ano'] = df_final['Ano'].astype('str')
# df_final.dtypes

## Coletando detalhadamente dados dos projetos apresentados

In [ ]:
headers = {'user-agent': 'Mozilla/5.0'}
detalhamento_propostas = requests.get('https://www.camarabotucatu.sp.gov.br/Consulta/Vereadores/',
                        headers= headers
                        )



In [ ]:
texto_detalhamento = BeautifulSoup(detalhamento_propostas.text)
itens_vereadores = texto_detalhamento.find_all('div', {'class': 'data-list-item data-list-striped data-list-hover'})

In [ ]:
# Pegando o link que leva a listagem de proposituras
lista_link_propositura = []
for i in range(len(itens_vereadores)):
    x = itens_vereadores[i].find_all('a')
    for j in range(len(x)):
        if x[j].text == 'Detalhadas':
            lista_link_propositura.append(x[j].get('href'))
    

In [ ]:
df_link_proposituras = pd.DataFrame()


# Pegar apenas a proposta de vereadores ativos no momento e não pegar Licenciados
lista_vereadores = list(df_vereadores['nome_vereador'])
lista_final_vereadores = []
for i in range(len(lista_vereadores)):
    if 'Licenciad' not in lista_vereadores[i]:
        lista_final_vereadores.append(lista_vereadores[i])

df_link_proposituras['vereador'] = lista_final_vereadores
df_link_proposituras['link_proposituras'] = lista_link_propositura
df_link_proposituras

In [ ]:
for indice, linha in df_link_proposituras.iterrows():
    valor_coluna1 = linha['vereador']
    valor_coluna2 = linha['link_proposituras']
    
    # faça alguma coisa com os valores de cada coluna da linha atual
    #print(f"O valor da coluna1 na linha {indice} é {valor_coluna1}")
    #print(f"O valor da coluna2 na linha {indice} é {valor_coluna2}")


In [ ]:
# #for i in range(len(lista_link_propositura)):
# link_detalhadas = 'https://www.camarabotucatu.sp.gov.br'+ lista_link_propositura[0]

# headers = {'user-agent': 'Mozilla/5.0'}
# respostas_detalhadas = requests.get(link_detalhadas,
#                         headers= headers
#                         )
# propostas_detalhadas = BeautifulSoup(respostas_detalhadas.text)

In [109]:
lista_dict = []
dict_data = {}
z = 0

for indice, linha in df_link_proposituras.iterrows():
    nome = linha['vereador']
    link = linha['link_proposituras']

    propostas_detalhadas = coleta_html('https://www.camarabotucatu.sp.gov.br'+ link)

    a = propostas_detalhadas.find_all('div', {'class': 'data-list-item data-list-striped data-list-hover'})

    for i in range(len(a)):
        tipo = a[i].h3
        z = 1
        
        for j in range(len(a[i].find_all('p'))):
            #print(a[i].find_all('p')[j].a.get('href') )
            print('Vereador: '+ nome + ' ' + tipo.text + ': '+str(z) )
            z= z + 1
            link = 'https://www.camarabotucatu.sp.gov.br'+a[i].find_all('p')[j].a.get('href')
            html_soup = coleta_html_detalhado(link, nome)

            if html_soup != 0:

                insere_link(link, nome)

                situacao = html_soup.find('strong', text='Situação:')
                
                if situacao:
                    data_strong = html_soup.find('strong', text='Data:')
                    
                    situacao_content = situacao.find_next_sibling(text=True).strip()
                    data_content = data_strong.find_next_sibling(text=True).strip()

                    data_content = datetime.strptime(data_content, '%d/%m/%Y')

                    if data_content.year >= 2021:
                    
                        dict_data = {
                            'Vereador': nome,
                            'Tipo': tipo,
                            'Situação': situacao_content,
                            'Data': data_content
                        }

                        lista_dict.append(dict_data)

                        time.sleep(10)

                        with open('dados.json', 'r+') as arquivo_json:
                            dados = json.load(arquivo_json)

                            dados.append(dict_data)

                            arquivo_json.seek(0)
                    
                    else: 
                        continue_outer_loop = True
                        break

                else:
                    print('')
            else:
                print("Já Existe:")
                continue_outer_loop = True
                break

    time.sleep(60)
        


Vereador: Abelardo Correspondências Recebidas (2): 1

Vereador: Abelardo Correspondências Recebidas (2): 2

Vereador: Abelardo Emendas (35): 1
Vereador: Abelardo Emendas (35): 2
Vereador: Abelardo Emendas (35): 3
Vereador: Abelardo Emendas (35): 4
Vereador: Abelardo Emendas (35): 5
Vereador: Abelardo Emendas (35): 6
Vereador: Abelardo Emendas (35): 7
Vereador: Abelardo Emendas (35): 8
Vereador: Abelardo Emendas (35): 9
Vereador: Abelardo Indicações (75): 1
Vereador: Abelardo Indicações (75): 2
Vereador: Abelardo Indicações (75): 3
Vereador: Abelardo Indicações (75): 4
Vereador: Abelardo Indicações (75): 5
Vereador: Abelardo Indicações (75): 6
Vereador: Abelardo Indicações (75): 7
Vereador: Abelardo Indicações (75): 8
Vereador: Abelardo Indicações (75): 9
Vereador: Abelardo Indicações (75): 10
Vereador: Abelardo Indicações (75): 11
Vereador: Abelardo Indicações (75): 12
Vereador: Abelardo Indicações (75): 13
Vereador: Abelardo Indicações (75): 14
Vereador: Abelardo Indicações (75): 15
V

KeyboardInterrupt: 

In [ ]:
# # criar o arquivo vazio
import json
with open('dados.json', 'w') as arquivo_json:
     json.dump([], arquivo_json)

In [ ]:
lista_dict

## Salvando no Banco de Dados

In [35]:
# conn = psycopg2.connect(
#     host=host,
#     port=port,
#     user=user,
#     password=password
# )

# #conn.set_isolation_level(0)
# #cur = conn.cursor()
# #cur.execute('CREATE DATABASE prefeitura')
# #conn.commit()
# #cur.close()
# #conn.close()



In [107]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cur = conn.cursor()

In [108]:
# # cur = conn.cursor()

# cur.execute('DELETE FROM links_proprosituras_vereadores')

# cur.execute("CREATE TABLE links_proprosituras_vereadores (nome_vereador VARCHAR(50), link VARCHAR(50) )")

# # #cur.execute("CREATE TABLE vereadores (nome_vereador VARCHAR(50), partido_vereador VARCHAR(50), link_vereador VARCHAR(200) );")
# # #cur.execute("CREATE TABLE proposituras (Tipo VARCHAR(50), Quantidade INTEGER, Ano VARCHAR(50), Vereador VARCHAR(50));")

# # #cur.execute("INSERT INTO tabela1 (coluna1, coluna2) VALUES (%s, %s);", (10, "texto"))
# # #cur.execute("INSERT INTO tabela2 (coluna1, coluna2) VALUES (%s, %s);", ("texto", 3.14))

# cur.execute("INSERT INTO links_proprosituras_vereadores (nome_vereador, link) VALUES (%s, %s);", ("Abelardo", 'LINK_ABELARDO'))

cur.execute("ALTER TABLE links_proprosituras_vereadores ALTER COLUMN link TYPE varchar(100)")
conn.commit()

cur.close()
conn.close()

In [ ]:
# from sqlalchemy import create_engine

# engine = create_engine('postgresql://postgres:'+password+'@'+host+':'+port+'/'+dbname)
# #df_vereadores.to_sql('vereadores', engine, if_exists='replace', index=False)
# df_final.to_sql('proposituras', engine, if_exists='replace', index=False)

In [ ]:
# ## TABELA DE IMAGENS DOS VEREADORES!!!
# cur = conn.cursor()

# cur.execute('''
#     CREATE TABLE imagens_vereadores (
#         vereador VARCHAR(50),
#         imagem BYTEA
#     );
# ''')

In [ ]:
# import os
# import psycopg2
# from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [ ]:
# diretorio = "C:/Users/thale/Documents/Estudos/Projeto Prefeitura"
# for arquivo in os.listdir(diretorio):
#     if arquivo.endswith(".jpg") or arquivo.endswith(".png"):
#         with open(os.path.join(diretorio, arquivo), "rb") as imagem_arquivo:
#             conteudo_imagem = imagem_arquivo.read()
#             arquivo1 = arquivo.split('.')[0]
#             cur.execute("INSERT INTO imagens_vereadores (vereador, imagem) VALUES (%s, %s)", (arquivo1, psycopg2.Binary(conteudo_imagem)))
#             conn.commit()
#         print(arquivo)

In [ ]:
# cur = conn.cursor()
# cur.execute('SELECT * FROM imagens_vereadores')

# results = cur.fetchall()
# for row in results:
#     print(row)

In [84]:
# cur = conn.cursor()
cur.execute('SELECT * FROM links_proprosituras_vereadores')

results = cur.fetchall()

df_teste = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])
print(df_teste.head())

for row in results:
    print(row)

Empty DataFrame
Columns: [nome_vereador, link]
Index: []


In [64]:
# # Execute a consulta SQL
# cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public' AND table_type='BASE TABLE';")

# # Obtenha os resultados da consulta
# tabelas = cur.fetchall()

# # Imprima as tabelas
# for tabela in tabelas:
#     print(tabela[0])


vereadores
proposituras
imagens_vereadores
links_proprosituras_vereadores


In [95]:
bbb = verifica_link('link', 'Abelardo')
bbb

Empty DataFrame
Columns: [link]
Index: []


[]